In [1]:
import pandas as pd
from scripts.data_genertion.consts import *

Added a column of the true length of the sequence according to the string length itself

Removed any rows with NaN inhibition cells

In [2]:
df = pd.read_csv('data_updated_18.5.csv')
df['true_length_of_seq'] = df[SEQUENCE].apply(len)
df = df.dropna(subset=[INHIBITION])

/tmp/ipykernel_2545/2826274691.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_updated_18.5.csv')


Filling Nan values

In [3]:
df_filled = df.copy()
for col in df.columns:
    if df[col].dtype == 'object':
        df_filled[col] = df[col].fillna("")
    else:
        df_filled[col] = df[col].fillna(0)

This part identify rows that may be the same experiment but with different modifications - in the type, location or pattern. This checks where there are groups of the same 'experiment_keys' that indicate the experiment, and search for more than one unique value at the 'columns_allowed_to_differ' where the modifications or linkage may be different to enable a comparison for only sequences that we can differentiate without considering the modifications. The added column is 'mod_scan', where 1 is a row that has at least one more row with the same experiment values and different modifications or locations of those.

In [4]:
experiment_keys = [SEQUENCE, 'Density(cells/well)', 'Transfection', 'Treatment_Period(hours)', 'ASO_volume(nM)']

columns_allowed_to_differ = [MODIFICATION, 'Location', 'Chemical_Pattern', 'Linkage', 'Linkage_Location', 'Smiles',
                             INHIBITION]

columns_to_compare = [col for col in df_filled.columns if col not in experiment_keys + columns_allowed_to_differ + ['ISIS']]

result_temp = []
for keys, group in df_filled.groupby(experiment_keys):
    # Check if any of the modification-related columns differ
    if group[[MODIFICATION, 'Location', 'Chemical_Pattern', 'Linkage', 'Linkage_Location', 'Smiles']].nunique().gt(1).any():
        # Check that all other fields (except inhibition + modifications) are the same
        compare = group[columns_to_compare].reset_index(drop=True)
        if compare.nunique().eq(1).all():
            result_temp.append(group)

if result_temp:
    final_df = pd.concat(result_temp)

    # Get the row indices of matched rows
    mod_scan_indices = final_df.index

    # Add new column to original dataframe
    df_filled['mod_scan'] = 0
    df_filled.loc[mod_scan_indices, 'mod_scan'] = 1

    print(df_filled[df_filled['mod_scan'] == 1])  # Optional: preview marked rows
else:
    df_filled['mod_scan'] = 0
    print("No matching ASO experiment groups with only modification-related differences.")


         ISIS Target_gene Cell_line  Density(cells/well) Transfection  \
2      540806       K-RAS      A431               5000.0  free uptake   
13     651966       K-RAS      A431               5000.0  free uptake   
15     651987       K-RAS      A431               5000.0  free uptake   
30     695867       K-RAS      A431               5000.0  free uptake   
31     695912       K-RAS      A431               5000.0  free uptake   
...       ...         ...       ...                  ...          ...   
37281  972044       APOL1      A431              11000.0  free uptake   
37283  972096       APOL1      A431              11000.0  free uptake   
37286  969448       APOL1      A431              11000.0  free uptake   
37287  971969       APOL1      A431              11000.0  free uptake   
37289  972189       APOL1      A431              11000.0  free uptake   

       ASO_volume(nM)  Treatment_Period(hours) Primer_probe_set  \
2              2000.0                       24      RTS3

In [8]:
df_filled.to_csv('data_asoptimizer_updated.csv', index=False)